# PAL & Plan‑Then‑Act Pipeline
LLM writes Python, we execute it, then ask LLM to explain. The Plan-and-Act Loop iteratively plans steps, executes them, and refines future actions using feedback.


## ⚠️ **SECURITY WARNING** ⚠️

**This notebook contains significant security risks and should only be used in a secure, isolated environment for educational purposes.**

### Security Risks:
- **Code Execution Risk**: This notebook uses `exec()` to run Python code generated by an LLM
- **No Sandboxing**: The executed code has full access to your system, files, and network
- **Potential for Malicious Code**: LLMs can generate harmful code including:
  - File system operations (read, write, delete files)
  - Network requests (data exfiltration, downloading malware)
  - System commands (potentially compromising your system)
  - Memory operations that could cause crashes

### Safety Recommendations:
1. **Only run in isolated environments** (containers, VMs, sandboxed environments)
2. **Never run on production systems** or systems with sensitive data
3. **Review generated code** before execution when possible
4. **Consider alternatives** like AST-based evaluation for simple expressions
5. **Implement input validation** and restricted execution environments for production use
6. **Consider sandboxing approaches** for production use:
   - Restrict Python builtins: `exec(code, {'__builtins__': {}}, {})`
   - Use RestrictedPython for more sophisticated restrictions
   - Run in containerized or virtualized environments

**For production applications, consider using safer alternatives like RestrictedPython, ast.literal_eval() for simple expressions, or containerized execution environments.**


In [ ]:
!pip -q install openai ipywidgets

In [ ]:
import os, openai, ipywidgets as w, contextlib, io, traceback, re, warnings
from IPython.display import display, Markdown
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

problem=w.Textarea(value='What is the factorial of 8?',description='Problem:',layout=w.Layout(width='100%',height='60px'))
run_btn=w.Button(description='Run PAL')
out=w.Output()

PAL_SYS='''You are an assistant that solves problems by first planning then writing Python.\nRespond with three fenced blocks plan/python/answer.''' 

last_plan=''; last_code=''

def run(_):
    global last_plan, last_code
    with out:
        out.clear_output()
        r=openai.ChatCompletion.create(model='gpt-4o-mini',
            messages=[{'role':'system','content':PAL_SYS},{'role':'user','content':problem.value}],
            temperature=0.3,max_tokens=400)
        txt=r.choices[0].message.content
        plan_match=re.search(r'```plan\n(.*?)```', txt, re.DOTALL)
        last_plan=plan_match.group(1).strip() if plan_match else ''
        display(Markdown('### LLM Response\n'+txt))
        if '```python' not in txt:
            print('No code block.');return
        code_match=re.search(r'```python\n(.*?)```', txt, re.DOTALL)
        if not code_match:
            print('No code block.');return
        code=code_match.group(1)
        last_code=code
        buf=io.StringIO()
        try:
            with contextlib.redirect_stdout(buf):
                warnings.warn(
                    "⚠️ SECURITY RISK: Executing LLM-generated code without sandboxing. "
                    "This is dangerous and should only be used in isolated environments.",
                    UserWarning,
                    stacklevel=2
                )
                exec(code, {})
            out_str=buf.getvalue().strip()
        except Exception:
            out_str=traceback.format_exc()
        print('--- Code Output ---');print(out_str)
        exp=openai.ChatCompletion.create(model='gpt-4o-mini',
            messages=[{'role':'system','content':'Explain the result.'},{'role':'user','content':out_str}],
            temperature=0.2,max_tokens=150)
        display(Markdown('### Explanation\n'+exp.choices[0].message.content))
run_btn.on_click(run)
display(w.VBox([problem, run_btn, out]))

In [ ]:
print('--- Plan ---')
print(last_plan)
print('--- Python Code ---')
print(last_code)
